In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
data=pd.read_csv('../merged/len3_ndb_agg_blp_DropNever.csv',encoding="shift-jis",index_col=0)
data.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,year,in_hospital,...,id_399.0,id_441.0,id_449.0,id_625.0,id_629.0,id_799.0,q_share,r_share,generic_share_q,generic_share_r
0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,292190.95,2014,1,...,0,0,0,0,0,0,0.000013,0.000007,0.242981,0.053692
1,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,128832.00,2014,1,...,0,0,0,0,0,0,0.000006,0.000003,0.242981,0.053692
2,114.0,解熱鎮痛消炎剤,661140020.0,インドメタシン坐剤 ５０ｍｇ,1145700J3012,19.3,0.0,106328.00,2014,1,...,0,0,0,0,0,0,0.000005,0.000003,0.242981,0.053692
3,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,658257.00,2014,0,...,0,0,0,0,0,0,0.000029,0.000016,0.242981,0.053692
4,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,418727.50,2014,0,...,0,0,0,0,0,0,0.000018,0.000010,0.242981,0.053692


In [3]:
data_est=data.dropna(subset=['lag_mean_price'])
ids=data_est.iloc[:,83:115]
X=data_est[["year"]+list(ids.columns)]
y=data_est["lag_mean_price"]
# X = sm.add_constant(X)  # 定数項を追加
model = sm.OLS(y, X).fit()

# モデルの要約を表示
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:         lag_mean_price   R-squared (uncentered):                   0.937
Model:                            OLS   Adj. R-squared (uncentered):              0.937
Method:                 Least Squares   F-statistic:                          4.580e+04
Date:                Mon, 04 Sep 2023   Prob (F-statistic):                        0.00
Time:                        05:54:14   Log-Likelihood:                     -6.0440e+05
No. Observations:              101193   AIC:                                  1.209e+06
Df Residuals:                  101160   BIC:                                  1.209e+06
Df Model:                          33                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [4]:
missing=data[data['lag_mean_price'].isnull()]
missing=missing[["year"]+list(ids.columns)]
# missing["const"]=1
predicted_values=model.predict(missing)
predicted_values

0        30.02194
1        30.02194
2        30.02194
3        30.02194
4        30.02194
          ...    
5100    130.18695
5101    130.18695
5102    130.18695
5103    130.18695
5104    130.18695
Length: 5105, dtype: float64

In [5]:
missing["lag_mean_price"]=predicted_values

In [6]:
data.update(missing)

In [7]:
# data.columns
data.to_csv('../merged/len3_ndb_agg_blp_imputed_lags.csv',encoding="shift-jis")